## Test interfacing with a remote DRP framework

Created: 2020-04-17, skwok

In [ ]:
import sys
import time
import glob

from interface import FrameworkInterface, Arguments, Event, Framework, ConfigClass


### Some help functions

In [ ]:
def print_pending_events (drpif):
    """
    Prints pending events, ie. events that are in the event queue.
    """
    if drpif.is_queue_ok():
        events = drpif.pending_events()
        for pe in events:
            print (pe)
    else:
        print ("Pending events: Queue not available", drpif.queue)

In [ ]:
def ingest_files (drpif, filelist):
    """
    Ingests 'next_file' events into the event queue.    
    """
    if drpif.is_queue_ok():
        for f in files:
            event = Event ("next_file", Arguments (name=f))
            print ("Adding ", f)
            drpif.add_event(event)
    else:
        print ("Ingest files: Queue not available", drpif.queue)    

### Get the DRPF interface to the queue manager

First, check is the queue is available, ie. if the queue manager process is already running.
If not, then start the queue manager process.

In [ ]:
cfg = ConfigClass ("multi_config.cfg")
drpf_IF = FrameworkInterface(cfg)

if not drpf_IF.is_queue_ok():
    print ("Staring the queue manager")
    res = drpf_IF.start_event_queue ()
    if res is None:
        print ("Faile to start queue")

#
# Check again
#
if not drpf_IF.is_queue_ok():
    print ("Failed to get remote event queue")
else:
    print ("Event queue ready")

### Add some data to the event queue

Check if DRPF queue manager is alive
then ingest some data files
and print the pending events


In [ ]:
files = glob.glob("../../keckdrpframework/unit_tests/test_files/*.fits")
ingest_files(drpf_IF, files)    

### Show how many events are in the queue

In [ ]:
print_pending_events(drpf_IF)

### Handle the events

At this point, there are two choices:
- Start a process or multiple processes to handle the events
- Invoke main_loop here 


The pipeline used in this notebook is the example pipeline included in the repository.

Run the following command in a xterm to process the data:
```
   cd ../../keckdrpframework/example
   sh runTest_harness2.sh -c config.cfg
```   
   

In [ ]:
#
# Choice #2, Invoking main_loop.
#
from keckdrpframework.examples.pipelines.fits2png_pipeline import Fits2pngPipeline
drpf = Framework(Fits2pngPipeline, "multi_config.cfg")
drpf.main_loop()

### Stop the queue manager

In [ ]:
if drpf_IF.is_queue_ok():
    drpf_IF.stop_event_queue()
    print ("Queue manager stopped")
else:
    print ("Queue manager already stopped")